In [1]:
from __future__ import print_function # Use a function definition from future version (say 3.x from 2.7 interpreter)
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
import cntk as C

%matplotlib inline

In [2]:

# Define the data dimensions
input_dim = 784
num_output_classes = 10

In [3]:
def create_reader(path, is_training, input_dim, num_label_classes):
    
    labelStream = C.io.StreamDef(field='labels', shape=num_label_classes, is_sparse=False)
    featureStream = C.io.StreamDef(field='features', shape=input_dim, is_sparse=False)
    
    deserializer = C.io.CTFDeserializer(path, C.io.StreamDefs(labels = labelStream, features = featureStream))
            
    return C.io.MinibatchSource(deserializer,
       randomize = is_training, max_sweeps = C.io.INFINITELY_REPEAT if is_training else 1)

In [4]:

input = C.input_variable(input_dim)
label= C.input_variable(num_output_classes)

** MODEL Creation **

In [5]:
def create_model(features):
    with C.layers.default_options(init=C.glorot_uniform()):
        return C.layers.Dense(10,activation=None)(features)
z=create_model(input/255.0)

In [6]:
loss=C.cross_entropy_with_softmax(z,label)
label_error=C.classification_error(z,label)

lr_schedule=C.learning_rate_schedule(0.2,C.UnitType.minibatch)
learner=C.sgd(z.parameters,lr_schedule)
trainer=C.Trainer(z,(loss,label_error),[learner])

In [7]:

data_dir=os.path.join("data","MNIST")
train_file = os.path.join(data_dir, "Train-28x28_cntk_text.txt")
test_file = os.path.join(data_dir, "Test-28x28_cntk_text.txt")

minibatch_size = 1000
num_samples_per_sweep = 60000
num_sweeps_to_train_with = 10
num_minibatches_to_train = (num_samples_per_sweep * num_sweeps_to_train_with) / minibatch_size


def moving_average(a, w=5):
    if len(a)< w:
        return a[:]
    return [val if idx < w else sum(a[(idx-w):idx])/w for idx, val in enumerate(a)]

def print_training_progress(trainer, mb, frequency, verbose=1):
    training_loss = "NA"
    eval_error = "NA"

    if mb%frequency == 0:
        training_loss = trainer.previous_minibatch_loss_average
        eval_error = trainer.previous_minibatch_evaluation_average
        if verbose: 
            print ("Minibatch: {0}, Loss: {1:.4f}, Error: {2:.2f}%".format(mb, training_loss, eval_error*100))
        
    return mb, training_loss, eval_error

In [8]:


reader_train=create_reader(train_file,True,input_dim,10)
input_map={
    label:reader_train.streams.labels,
    input:reader_train.streams.features
}
training_progress_output_freq=1000

plotdata={"batchsize":[],"loss":[],"error":[]}
for i in range(int(num_minibatches_to_train)):
    
    # Read a mini batch from the training data file
    data = reader_train.next_minibatch(minibatch_size, input_map = input_map)
    
    trainer.train_minibatch(data)
    batchsize, loss, error = print_training_progress(trainer, i, training_progress_output_freq, verbose=1)
    
    if not (loss == "NA" or error =="NA"):
        plotdata["batchsize"].append(batchsize)
        plotdata["loss"].append(loss)
        plotdata["error"].append(error)



Minibatch: 0, Loss: 2.3294, Error: 85.00%
Minibatch: 1000, Loss: 0.2896, Error: 5.00%
Minibatch: 2000, Loss: 0.1582, Error: 5.00%
Minibatch: 3000, Loss: 0.1805, Error: 5.00%
Minibatch: 4000, Loss: 0.3868, Error: 15.00%
Minibatch: 5000, Loss: 0.1732, Error: 5.00%
Minibatch: 6000, Loss: 0.1461, Error: 5.00%
Minibatch: 7000, Loss: 0.0304, Error: 0.00%
Minibatch: 8000, Loss: 0.1371, Error: 5.00%
Minibatch: 9000, Loss: 0.1160, Error: 5.00%
Minibatch: 10000, Loss: 0.1650, Error: 5.00%
Minibatch: 11000, Loss: 0.2790, Error: 10.00%
Minibatch: 12000, Loss: 0.0373, Error: 0.00%
Minibatch: 13000, Loss: 0.4425, Error: 25.00%
Minibatch: 14000, Loss: 0.1042, Error: 10.00%
Minibatch: 15000, Loss: 0.0833, Error: 0.00%
Minibatch: 16000, Loss: 0.1726, Error: 10.00%
Minibatch: 17000, Loss: 0.2032, Error: 5.00%
Minibatch: 18000, Loss: 0.4068, Error: 10.00%
Minibatch: 19000, Loss: 0.1409, Error: 0.00%
Minibatch: 20000, Loss: 0.2995, Error: 10.00%
Minibatch: 21000, Loss: 0.1359, Error: 0.00%
Minibatch: 2200

_PLOTTING_

In [ ]:
# Read the training data
reader_test = create_reader(test_file, False, input_dim, num_output_classes)

test_input_map = {
    label  : reader_test.streams.labels,
    input  : reader_test.streams.features,
}

# Test data for trained model
test_minibatch_size = 512
num_samples = 10000
num_minibatches_to_test = num_samples // test_minibatch_size
test_result = 0.0

for i in range(num_minibatches_to_test):
    
    # We are loading test data in batches specified by test_minibatch_size
    # Each data point in the minibatch is a MNIST digit image of 784 dimensions 
    # with one pixel per dimension that we will encode / decode with the 
    # trained model.
    data = reader_test.next_minibatch(test_minibatch_size,
                                      input_map = test_input_map)

    eval_error = trainer.test_minibatch(data)
    test_result = test_result + eval_error

# Average of evaluation errors of all test minibatches
print("Average test error: {0:.2f}%".format(test_result*100 / num_minibatches_to_test))

Average test error: 7.87%


In [ ]:
out=C.softmax(z)
reader_eval = create_reader(test_file, False, input_dim, num_output_classes)

eval_minibatch_size = 25
eval_input_map = {input: reader_eval.streams.features,
                 label:reader_eval.streams.labels} 

data = reader_eval.next_minibatch(eval_minibatch_size, input_map = eval_input_map)

img_label = data[label].asarray()
img_data = data[input].asarray()
predicted_label_prob = [out.eval(img_data[i]) for i in range(len(img_data))]


pred=[np.argmax(predicted_label_prob[i]) for i in range(len(predicted_label_prob))]
actual_label=[np.argmax(img_label[i]) for i in range(len(img_label))]

print(pred[:25])
print(actual_label)

i=12
plt.imshow(img_data[12].reshape(28,28),cmap="grey_r")
print(pred[i])